In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
gray_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_GRAYSCALE) #이미지를 흑백이미지로 읽음
plt.imshow(gray_lena_img, cmap='gray') #흑백의 원본 사진 출력


In [ ]:
# np.uint8 을 이용한 정규화 부분 실험
x=np.array([-1])
a=np.uint8(x)
print(a) # 넘파이 배열 형식에 -1을 할당후 진행하면 255가 나옴



In [ ]:
# sobel filter

def sobel_filter(img):
    img_h, img_w = img.shape
    sobel_x_filter = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    sobel_y_filter = np.array([[-1, -2, -1],
                               [ 0,  0,  0],
                               [ 1,  2,  1]])

    sobel_x_img = np.zeros((img.shape[0],img.shape[1])) # x축 방향 검출에지를 담을 배열
    sobel_y_img = np.zeros((img.shape[0],img.shape[1])) # y축 방향 검출에지를 담을 배열

    filter_half = sobel_x_filter.shape[0] // 2# 컨볼루션 시작 범위 인덱스

    for y in range(filter_half, img_h - filter_half):
        for x in range(filter_half, img_h - filter_half):
            img_patch = img[x - filter_half : x + filter_half + 1, y - filter_half : y + filter_half + 1]

            sobel_x_img[x, y] = np.sum(img_patch * sobel_x_filter) # x방향 소벨 필터 진행
            sobel_y_img[x, y] = np.sum(img_patch * sobel_y_filter) # y방향 소벨 필터 진행

    total_sobel_img = np.sqrt(sobel_x_img**2, sobel_y_img**2)
    #total_sobel_img -= total_sobel_img.min() # 최소값을 0으로 이동--음수값을 갖는 픽셀값도 모두 올바르게 정규화 가능
    total_sobel_img = np.uint8(total_sobel_img) # np.unit8() 이용하여 정규화 진행??
    #total_sobel_img *= 255 / total_sobel_img.max() # 모든 픽셀값을 0~255 사이로 정규화

    # sobel_x 필터링 이미지와 sobel_y 필터링 이미지도 정규화 진행
    sobel_x_img -= sobel_x_img.min()
    sobel_x_img *= 255 / sobel_x_img.max()
    sobel_y_img -= sobel_y_img.min()
    sobel_y_img *= 255 / sobel_y_img.max()

    return total_sobel_img, sobel_x_img, sobel_y_img

sobel_total, sobel_x, sobel_y = sobel_filter(gray_lena_img)

plt.figure(figsize=(15, 10))
plt.subplot(1,3,1)
plt.imshow(sobel_x, cmap='gray')
plt.title('sobel_x')
plt.subplot(1,3,2)
plt.imshow(sobel_y, cmap='gray')
plt.title('sobel_y')
plt.subplot(1,3,3)
plt.imshow(sobel_total, cmap='gray')
plt.title('sobel_total')
plt.show()
'''
# 결과값을 0~255 범위 내에서 표현--> np.unit8 활용
sobel_img_x_final = np.uint8(np.abs(sobel_img_x))
sobel_img_y_final = np.uint8(np.abs(sobel_img_y))
'''

In [ ]:
def convolution(image, filter):

  image_height, image_width = image.shape
  filter_height, filter_width = filter.shape
  padding = filter_height // 2
  filtered_image = np.zeros_like(image) #새로운 복사 이미지

  #반복문을 수행하면서 패딩&컨볼루션 진행
  #가장자리 부분은 계산하기 귀찮기에 아에 제외하면서 진행
  for y in range(padding, image_height - padding):
    for x in range(padding, image_width - padding):
      image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
      filtered_image[y, x] = np.sum(image_patch * filter)

  return filtered_image


In [ ]:
sobel_x_filter = np.array([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]])

sobel_y_filter = np.array([[-1, -2, -1],
                            [ 0,  0,  0],
                            [ 1,  2,  1]])
# 정규화 계산 진행을 위해 .astype(np.float64)으로 형변환
sobel_img_x = convolution(gray_lena_img, sobel_x_filter).astype(np.float64)
sobel_img_y = convolution(gray_lena_img, sobel_y_filter).astype(np.float64)


sobel_img_x -= sobel_img_x.min()
sobel_img_x *= 255 / sobel_img_x.max()
sobel_img_y -= sobel_img_y.min()
sobel_img_y *= 255 / sobel_img_y.max()

# 정규화 진행 이후 unit8로 다시 변환
sobel_img_x = sobel_img_x.astype(np.uint8)
sobel_img_y = sobel_img_y.astype(np.uint8)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(sobel_img_x,cmap='gray')
plt.title('sobel_img_x')
plt.subplot(1,2,2)
plt.imshow(sobel_img_y,cmap='gray')
plt.title('sobel_img_y')
plt.show()


In [ ]:
# 소벨 X와 Y를 결합하여 최종 엣지 맵 생성
sobel_combined = np.sqrt(sobel_img_x**2 + sobel_img_y**2)
plt.imshow(sobel_combined, cmap='gray')
plt.show()

In [ ]:
check_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/check_img.jpg', cv2.COLOR_BGR2RGB) #이미지를 bgr이미지를 rgb 이미지로 읽음
rgb_check_img = cv2.cvtColor(check_img,cv2.COLOR_BGR2RGB)
gray_check_img = cv2.cvtColor(check_img,cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(rgb_check_img) #rgb의 원본 사진 출력
plt.title('rgb img')

plt.subplot(1,2,2)
plt.imshow(gray_check_img, cmap='gray') #흑백 사진 출력
plt.title('gray img')



In [ ]:
# 2번 교수님 버전 양방향 보간법
# 흑백 이미지 버전
def bilinear_interpolation_function(img, s_factor):
    h,w = img.shape
    sh,sw = int(np.floor(h*s_factor)), int(np.floor(w*s_factor)) # 소수를 곱하는 경우를 대비해서 소수점 버림 (np.floor 항상 낮은 값으로 버림 진행)
    out_img = np.zeros((sh,sw))
    # source : x,y
    # target : sx, sy

    for sy in range(sh):
        for sx in range(sw):

            y = int(sy / s_factor)
            x = int(sx / s_factor)

            a = sy / s_factor - y
            b = sx / s_factor - x

            if y+1 >= h or x+1 >= w:  #오버플로우 방지 해당영역을 만났을 때 스킵하고 진행
                continue
            out_img[sy,sx] = (1-a)*( (1-b) * img[y,x] + b * img[y+1,x]) + a * ( (1-b) * img[y,x+1] + b * img[y+1,x+1]) # 보간법 진행

    return out_img


scaling_img = bilinear_interpolation_function(gray_check_img, 4)
plt.imshow(scaling_img,cmap='gray')



In [ ]:
# 2) Design an image scaling function using bilinear operation, and compare it to cv2.resize() function.
# Input : image, scaling factor
# Output : scaled image
# 양선형 보간 진행

def bilinear_scaling(image, scaling_factor):
    #height, width = image.shape[:2] # 흑백 이미지 처리용 코드
    height, width, channels = image.shape # rgb 이미지 처리용 코드--채널값 추가
    new_height = int(height * scaling_factor)
    new_width = int(width * scaling_factor)
    #scaled_image = np.zeros((new_height, new_width), dtype=np.uint8) # 흑백 이미지 처리용 코드
    scaled_image = np.zeros((new_height, new_width, channels), dtype=np.uint8) # 채널값 추가


    for y in range(new_height):
        for x in range(new_width):
            original_x = x / scaling_factor
            original_y = y / scaling_factor

            x1 = int(original_x)
            x2 = min(x1 + 1, width - 1)
            y1 = int(original_y)
            y2 = min(y1 + 1, height - 1)

            # rgb이미지용 양방향 보간 코드 -- 채널별로 진행
            for c in range(channels): # 각 채널마다 보간을 진행(R, G, B)
                q11 = image[y1, x1, c]
                q12 = image[y2, x1, c]
                q21 = image[y1, x2, c]
                q22 = image[y2, x2, c]

                scaled_image[y, x, c] = int(q11 * (x2 - original_x) * (y2 - original_y) +
                                           q21 * (original_x - x1) * (y2 - original_y) +
                                           q12 * (x2 - original_x) * (original_y - y1) +
                                           q22 * (original_x - x1) * (original_y - y1))

            '''
            # 흑백이미지용 양방향 보간 코드--채널 부분이 없음
            # Bilinear interpolation
            q11 = image[y1, x1]
            q12 = image[y2, x1]
            q21 = image[y1, x2]
            q22 = image[y2, x2]

            scaled_image[y, x] = int(q11 * (x2 - original_x) * (y2 - original_y) +
                                   q21 * (original_x - x1) * (y2 - original_y) +
                                   q12 * (x2 - original_x) * (original_y - y1) +
                                   q22 * (original_x - x1) * (original_y - y1))
            '''

    return scaled_image

# Scaling Factor
scaling_factor = 0.5

# Bilinear scaling
bilinear_scaled_image = bilinear_scaling(rgb_check_img, scaling_factor)


# OpenCV 내장함수 resize 사용
cv2_scaled_image = cv2.resize(rgb_check_img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_LINEAR)


plt.figure(figsize=(15, 10))

plt.subplot(1, 3, 1)
plt.imshow(rgb_check_img, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(bilinear_scaled_image, cmap='gray')
plt.title('Bilinear Scaled Image')

plt.subplot(1, 3, 3)
plt.imshow(cv2_scaled_image, cmap='gray')
plt.title('cv2.resize() Scaled Image')

plt.show()


In [ ]:

def bilinear_interpolation(img, scale_x, scale_y):
    h, w = img.shape[0], img.shape[1]
    new_h, new_w = int(h * scale_y), int(w * scale_x)
    #new_img = np.zeros((new_h, new_w), dtype=np.uint8) # 흑백이미지 처리용
    new_img = np.zeros((new_h, new_w, 3), dtype=np.uint8) # rgb이미지 처리용, 채널값 3 추가


    for i in range(new_h):
        for j in range(new_w):
            # 원래 이미지에서 대응되는 좌표 계산
            x = j / scale_x
            y = i / scale_y

            # 가장 가까운 네 개의 픽셀 좌표 찾기
            x1, y1 = int(x), int(y)
            x2, y2 = min(x1 + 1, w - 1), min(y1 + 1, h - 1)

            # 픽셀 값 가져오기
            Q11 = img[y1, x1]
            Q21 = img[y1, x2]
            Q12 = img[y2, x1]
            Q22 = img[y2, x2]

            # 보간 수행
            R1 = (x2 - x) * Q11 + (x - x1) * Q21
            R2 = (x2 - x) * Q12 + (x - x1) * Q22
            P = (y2 - y) * R1 + (y - y1) * R2

            new_img[i, j] = np.clip(P, 0, 255)  # 값이 0~255 범위 유지

    return new_img

# Example usage and comparison:
scaling_factor = 5  # Example scaling factor

# 2배 확대
resized_image = bilinear_interpolation(rgb_check_img, 5, 5)

# OpenCV's resize function
cv2_scaled_image = cv2.resize(rgb_check_img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_LINEAR)

plt.figure(figsize=(15,10))
plt.subplot(1,3,1)
plt.imshow(rgb_check_img,cmap='gray')
plt.title('Original img')
plt.subplot(1,3,2)
plt.imshow(resized_image,cmap='gray')
plt.title('Resized img')
plt.subplot(1,3,3)
plt.imshow(cv2_scaled_image,cmap='gray')
plt.title('cv2.resize() img')
plt.show()

In [ ]:
# OpenCV's resize function
cv2_scaled_image = cv2.resize(rgb_check_img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_LINEAR)
plt.imshow(cv2_scaled_image,cmap='gray')
plt.title('cv2.resize() img')
plt.show()

In [ ]:
# Bicubic interpolation (컬러 이미지용)
def bicubic_interpolation(img, target_shape):
    if len(img.shape) == 3:  # 컬러 이미지일 경우
        h, w, c = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, c), dtype=img.dtype)
        for channel in range(c):
            scale_x = w / new_w
            scale_y = h / new_h
            for i in range(new_h):
                for j in range(new_w):
                    x = j * scale_x
                    y = i * scale_y
                    x0 = int(x)
                    y0 = int(y)
                    # Bicubic interpolation을 위해 주변 4x4 픽셀 사용
                    x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
                    y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
                    dx = x - x0
                    dy = y - y0
                    # Bicubic 커널 계산
                    def cubic(t):
                        t = np.abs(t)
                        if t <= 1:
                            return 1 - 2*t**2 + t**3
                        elif t <= 2:
                            return -t**3 + 5*t**2 - 8*t + 4
                        return 0
                    value = 0
                    for m in range(4):
                        for n in range(4):
                            weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                            value += weight * img[y_indices[n], x_indices[m], channel]
                    output[i, j, channel] = np.clip(value, 0, 255)
    else:  # 흑백 이미지일 경우
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
        scale_x = w / new_w
        scale_y = h / new_h
        for i in range(new_h):
            for j in range(new_w):
                x = j * scale_x
                y = i * scale_y
                x0 = int(x)
                y0 = int(y)
                x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
                y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
                dx = x - x0
                dy = y - y0
                def cubic(t):
                    t = np.abs(t)
                    if t <= 1:
                        return 1 - 2*t**2 + t**3
                    elif t <= 2:
                        return -t**3 + 5*t**2 - 8*t + 4
                    return 0
                value = 0
                for m in range(4):
                    for n in range(4):
                        weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                        value += weight * img[y_indices[n], x_indices[m]]
                output[i, j] = np.clip(value, 0, 255)
    return output
